In [1]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import transformers

print(transformers.__version__)

4.30.2


In [5]:
from datasets import load_dataset

datasets = load_dataset("text", data_files={"train": "big-cleanMixedPresFinalOne.txt", "validation": "testData.txt"})

  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
datasets["train"][10]

{'text': ' You have of course read [something like this](_URL_0_) in your quest, have you not? You have been on a quest for knowledge.'}

In [7]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [8]:
show_random_elements(datasets["train"])

,text
0,"Let me see if I understand you correctly: your radio is playing signals from some station, and as you walk away from it, without moving the radio itself, you hear more static? There is a radio present."
1,"Hi ElusiveUsama thank you for submitting to /r/Askscience. ElusiveUsama submitted a question to /r/Askscience. **If your post is not flaired it will not be reviewed. Flairing posts is a common practice on online forums. ** Please add flair to your post. You have previously made a post. Your post will be removed permanently if flair is not added within one hour. Flair is a necessary component for posts on this platform. You can flair this post by replying to this message with your flair choice. Flairing a post is a common action on this platform. It must be an exact match to one of the following flair categories and contain no other text: 'Computing', 'Economics', 'Human Body', 'Engineering', 'Planetary Sci. The categories listed are the only ones that exist in the field. ', 'Archaeology', 'Neuroscience', 'Biology', 'Chemistry', 'Medicine', 'Linguistics', 'Mathematics', 'Astronomy', 'Psychology', 'Paleontology', 'Political Science', 'Social Science', 'Earth Sciences', 'Anthropology', 'Physics' ___ Your post is not yet visible on the forum and is awaiting review from the moderator team. The field of study mentioned in the sentence is a highly specialized and technical one. Your question may be denied for the following reasons, * **Have you searched for your question on** [**AskScience**](_URL_4_ & sort=relevance & restrict_sr=on & t=all) **or on** [**Google**](_URL_6_)**? You have a question that you want answered. ** - Common questions, or questions covered in the [**FAQ**](_URL_0_), will be rejected. There is a frequently asked questions section available. * **Are you asking for** [**medical advice**](_URL_2_) **or does your post contain personal medical information? There is a possibility that the person in question is experiencing a medical issue. ** - These questions, even innocuous ones should be between you and your doctor. You have a medical condition that requires you to ask questions to your doctor. * **Is your post speculative or hypothetical? There is a possibility that the post is speculative or hypothetical. ** - Questions involving unphysical what ifs or imaginary situations requiring guesses and speculation are best for /r/AskScienceDiscussion. There are many unphysical what ifs and imaginary situations that people ask about on /r/AskScienceDiscussion. There are more restrictions on what kind of questions are suitable for /r/AskScience, the above are just some of the most common. /r/AskScience is a highly regulated platform. While you wait, check out the forum [**Posting Guidelines**](_URL_1_) on asking questions as well as our [**User Help Page**](_URL_5_). There are people waiting for something. Please wait several hours before messaging us if there is an issue, moderator mail concerning recent submissions will be ignored. There have been recent submissions. ___*I am a bot, and this action was performed automatically. The speaker is not human. Please contact the* [*moderators of this subreddit*](_URL_3_) *if you have any questions or concerns. The subreddit has moderators. * I'm sorry, but there is no sentence indicated by angel brackets in your prompt."
2,"Some types of glue contains a solvent that evaporates. Glue is commonly used in various industries. After the solvent has evaporated, the glue hardens and binds the pieces together. The solvent was present before it evaporated. An example would be white glue. White glue is a common household item. Other types undergo a chemical reaction that binds the pieces together. Other materials do not undergo a chemical reaction that binds the pieces together. Superglue is an example of this. Superglue is commonly used in industrial settings. Yet other types of glue are heated to flow easily to fill in the gap between objects, and hardens when cooled. There 

In [9]:
model_checkpoint = "distilgpt2"

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [11]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [12]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

In [13]:
tokenized_datasets["train"][1]

{'input_ids': [921,
  389,
  2192,
  3375,
  546,
  21047,
  16751,
  351,
  362,
  11,
  19,
  12,
  35,
  488,
  4685,
  47806,
  23536,
  330,
  5139,
  7408,
  14,
  17,
  11,
  19,
  12,
  35,
  355,
  262,
  4075,
  18734,
  13,
  1318,
  318,
  257,
  1029,
  14955,
  326,
  21047,
  16751,
  351,
  362,
  11,
  19,
  12,
  35,
  488,
  4685,
  47806,
  23536,
  330,
  5139,
  7408,
  14,
  17,
  11,
  19,
  12,
  35,
  355,
  262,
  4075,
  18734,
  389,
  852,
  6693,
  13,
  362,
  11,
  19,
  12,
  35,
  2499,
  416,
  12034,
  3649,
  27506,
  259,
  3227,
  13,
  47105,
  259,
  3227,
  318,
  1877,
  1231,
  262,
  779,
  286,
  362,
  11,
  19,
  12,
  35,
  13,
  47105,
  1040,
  389,
  257,
  1398,
  286,
  4618,
  3349,
  17199,
  326,
  287,
  3376,
  6867,
  7719,
  2685,
  3349,
  13,
  39269,
  2314,
  1663,
  1231,
  27506,
  1040,
  13,
  2102,
  11,
  262,
  625,
  25493,
  286,
  27506,
  259,
  4073,
  416,
  362,
  11,
  19,
  12,
  35,
  5640,
  39483,
  54

In [14]:
# block_size = tokenizer.model_max_length
block_size = 128

In [15]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [16]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

In [17]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

' situation. Again, the question becomes "how long will this take? There is a task or project that needs to be completed. ", which depends on the properties of the ring. The ring has unique properties that set it apart from other rings. Immediately after the formation of a ring (however that happens), they needn\'t be aligned. A ring has been formed. But eventually, they\'ll be aligned. There was a previous misalignment. Rings can only effectively extend out to the Roche Radius - beyond that, their self-gravity will result in them accumulating into moons. The Roche Radius is a well-known astronomical term.'

In [18]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

In [19]:
from transformers import Trainer, TrainingArguments

In [20]:
!pip install accelerate -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-eli5",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [23]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.556700,3.940048
2,3.337700,3.977760
3,3.271000,3.990552


TrainOutput(global_step=2883, training_loss=3.365551766280784, metrics={'train_runtime': 677.8388, 'train_samples_per_second': 34.021, 'train_steps_per_second': 4.253, 'total_flos': 753220545675264.0, 'train_loss': 3.365551766280784, 'epoch': 3.0})

In [24]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 54.08


In [25]:
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.6 MB/s eta 0:00:00


In [27]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"


In [28]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.6 MB/s eta 0:00:00


In [31]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from bert_score import score

tokenizer = AutoTokenizer.from_pretrained('distilgpt2')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

test_data = []

with open('testData.txt', 'r', encoding='utf-8') as file:
    test_data = [line.strip() for line in file if line.strip()]

predictions = []
refs = []

# Function to split string at the middle but not cut a word into half
def split_string(s):
    if ' ' not in s:  # If the text doesn't contain a space, return the text as is
        return s, ''
    else:
        midpoint = len(s) // 2  # Integer division to get the middle index
        while midpoint < len(s) - 1 and s[midpoint] != ' ':
            midpoint += 1
        return s[:midpoint], s[midpoint:]

i = 0
# Generate predictions
for line in test_data[:200]:
    prompt, expected_completion = split_string(line)
    inputs = tokenizer.encode(prompt, return_tensors='pt').to(device)
    outputs = model.generate(inputs, max_length=250, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    generated_completion = tokenizer.decode(outputs[0])[len(tokenizer.decode(inputs[0])):]
    print(generated_completion.strip())
    print(expected_completion.strip())
    predictions.append(generated_completion.strip())
    refs.append(expected_completion.strip())
    print(i)
    i = i + 1

print(len(hyps))
print(len(predictions))
print(len(refs))
# Calculate BERTScore
hyps = [pred for pred in predictions if pred]

P, R, F1 = score(predictions, refs, lang='en', model_type='bert-base-uncased')

print(f"Mean F1 BERTScore: {F1.mean().item()}")


our brain. Pain and pleasure are both literally "just in your mind". The brain is capable of processing and processing emotions. So, if you're thinking of a "painless" brain, then you're not thinking of a "painless" brain. There is a brain that is capable of processing and processing emotions. So, if you're thinking of a "painless" brain, then you're not thinking of a "painless" brain. There is a specific situation that needs to be addressed. So, if you're thinking of a "painless" brain, then you're not thinking of a "painless" brain. There is a specific situation that needs to be addressed. So, if you're thinking of a "painless" brain, then you're not thinking of a "painless" brain. There is a specific situation that needs to be addressed. So, if you're thinking of a "painless" brain, then you're not thinking of a "painless" brain. There
our brains. The pain and pleasure areas lie fairly closely together in the brain. Neurons from one "area" can grow across to the other, or get crosse

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Mean F1 BERTScore: 0.331216961145401
